# Анализ спроса на самокаты.

Данные о поездках на самокатах представляют ценную информацию для понимания паттернов спроса и предпочтений пользователей. В нашем исследовании мы анализируем информацию о поездках на самокатах, включая дату и время начала и окончания поездки, местоположение старта и финиша, а также пройденное расстояние. Целью анализа является выявление основных трендов, факторов влияющих на спрос и закономерностей в использовании самокатов в различных районах города.

## 1. Загрузка библиотек и глобальных переменны

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta
import seaborn as sns
import Levenshtein
import matplotlib.pyplot as plt
import matplotlib.dates
from scipy.stats import shapiro
from datetime import datetime
from scipy.stats import f_oneway
DIR = '/content/drive/MyDrive/'
DIR = './'

### 1.1. Пользовательские функции

In [ ]:
def check_data(data_df):
    print('\033[1m' + 'Изучим исходные данные' + '\033[0m')
    print(data_df.info())
    # print(data_df.shape)

    missed_cells = data_df.isnull().sum().sum(
    )/(data_df.shape[0]*(data_df.shape[1]-1))
    missed_rows = sum(data_df.isnull().sum(axis=1) > 0)/data_df.shape[0]
    print('\033[1m' + '\nПроверка пропусков' + '\033[0m')
    print('Количество пропусков: {:.0f}'.format(data_df.isnull().sum().sum()))
    print('Доля пропусков: {:.1%}'.format(missed_cells) + '\033[0m')
    print('Доля строк содержащих пропуски: {:.1%}'.format(missed_rows))

    # Проверим дубликаты
    print('\033[1m' + '\nПроверка на дубликаты' + '\033[0m')
    print('Количество полных дубликатов: ', data_df.duplicated().sum())

    # Посмотрим на сами данные
    print('\033[1m' + '\nПервые пять строк датасета' + '\033[0m')
    display(data_df.head(10))  # tail(7)

    print('\033[1m' + '\nОписание количественных данных:' + '\033[0m')
    display(data_df.describe().T)

    print('\033[1m' + '\nОписание категориальных данных:' + '\033[0m')
    display(data_df.describe(include='object').T)

    print('\033[1m' + '\nВывод уникальных значений по каждому категориаьному признаку:' + '\033[0m')
    df_object = data_df.select_dtypes(include='object').columns

    for i in df_object:
        print('\033[1m' + '_' + str(i) + '\033[0m')
        display(data_df[i].value_counts())


plt.show()




In [ ]:
def find_similar_strings(df_main):
    implicit_duplicates = []
    reference_string = None
    threshold_distance = 3  # Пороговое значение для считывания строки как дубликата
    for index, string in df_main.iterrows():
        if reference_string is None:
            reference_string = string
            continue
        distance = Levenshtein.distance(reference_string, string)
        if distance < threshold_distance:
            implicit_duplicates.append((reference_string, string))
        else:
            reference_string = string
    return implicit_duplicates

In [ ]:
def cat_graph(df, cat_feat):
    '''
    Функция отрисовки круговых диаграмм для категориальных переменных.
    На вход: исходная таблица и список категориальных переменных.
    На выходе: графики
    '''

    cols = 2
    rows = int(np.ceil(len(cat_feat) / cols))

    fig, axs = plt.subplots(rows, cols, figsize=(25, 20))
    plt.tight_layout()

    count = -1
    for i in range(rows):
        for x in range(cols):
            count += 1
            col = cat_feat[count]
            df1 = pd.DataFrame(df.groupby([col])[col].count())
            axs[i, x].pie(x=df1[col],
                          labels=df1.index,
                          autopct='%1.1f%%',)
            axs[i, x].title.set_text(str(col))

    plt.suptitle('Круговые диаграммы категориальных признаков',
                 fontsize=20, y=1.03)

    plt.show()

In [ ]:
def plot_hist(data, col_column):
    '''
    Функция отрисовки гистограмм и ящика с усами для количесвтенных переменных.
    На вход: исходная таблица и список количественных переменных.
    На выходе: графики
    '''
    rows = len(col_column)
    f, ax = plt.subplots(rows,2, figsize=(8, 15))
    f.tight_layout()
    f.set_figheight(30)
    f.set_figwidth(14)
    plt.rcParams.update({'font.size': 18})
    
    for i, col in enumerate(col_column):         
        sns.histplot(data[col], kde=True, bins=24, ax = ax[i, 0])                    
        sns.boxplot(data[col], ax = ax[i, 1])

        ax[i, 0].set_xlabel(col)
        ax[i, 1].set_xlabel(col)
        ax[i, 0].set_ylabel('Количество')
    plt.suptitle("Гистограмма и ящик с усами для количесвтенных данных", fontsize=22, y=1.01)
    plt.show()

In [ ]:
def normal_sum_test(x, Ptest):
    """
    Функция проверяет нормальность/ненормальность распределения
    по сумме 3-х тестов: Шапиро, Андерсона-Дарлинга, Харке-Бера

    На выходе:
    1 - ненормальное распределение
    0 - нормальное распредление

    Принцип большинства заложен.
    Внутри есть функция расчёта критерия Андерсона, исходя из уровня значимсоти
    """

    def anderson_chois_sig(A, Ptest):
        if Ptest == 0.05:
            ander = A[2]
        elif Ptest == 0.01:
            ander = A[4]
        return ander

    def normalnost_anderson(x, Ptest):
        A2, crit, sig = anderson(x, dist='norm')
        ad_pass = (A2 < crit)
        norm = anderson_chois_sig(ad_pass, Ptest)
        if norm == False:
            return 1
        return 0
   # print(x)
    p_shapiro = shapiro(x)[1]
    p_jarque = jarque_bera(x)[1]

    if p_shapiro < Ptest:
        p_shapiros = 1
    else:
        p_shapiros = 0

    if p_jarque < Ptest:
        p_jarques = 1
    else:
        p_jarques = 0

    p_anderson = normalnost_anderson(x, Ptest)  # 1 - ненормальное, 0 - нормальное

    p_sum = p_shapiros + p_anderson + p_jarques

    if (p_sum > 1):
        return 1
    else:
        return 0

### 1.2. Загрузка данных

In [ ]:
df_main = pd.read_csv("rides.csv")

In [ ]:
# Проверка отработанного кода
df_main.head(10)

In [ ]:
df_weather = pd.read_csv("weather.csv")

In [ ]:
# Проверка отработанного кода
df_weather.head(10)

## 2. Осмотр данных

### 2.1 Основаня таблица 

In [ ]:
# ОПИСАТЕЛЬНЫЙ АНАЛИЗ
check_data(df_main)

In [ ]:
# ГРАФИЧЕСКИЙ АНАЛИЗ количественных данных данных
num_features = df_main.select_dtypes(exclude=[object]).columns

# Проверка
num_features

*Вывод: Отсутвует время начала и время конца поездки*

In [ ]:
plot_hist(df_main, df_main.select_dtypes(exclude=[object]).columns)

In [ ]:
# ГРАФИЧЕСКИЙ АНАЛИЗ категориальных данных
cat_features = df_main.select_dtypes(include=[object]).columns

# Проверка
cat_features

*Вывод: Есть лишние данные в виде даты старта и даты окончания*

### 2.2 Таблица с погодой

In [ ]:
# ОПИСАТЕЛЬНЫЙ АНАЛИЗ
check_data(df_weather)

In [ ]:
# ГРАФИЧЕСКИЙ АНАЛИЗ количественных данных данных
num_features = df_weather.select_dtypes(exclude=[object]).columns

# Проверка
num_features

In [ ]:
# ГРАФИЧЕСКИЙ АНАЛИЗ категориальных данных
cat_features = df_weather.select_dtypes(include=[object]).columns

# Проверка
cat_features

*Вывод: Не правильно назначенны типы данных*

### 2.3 Выводы

**Основная таблица:** 
* Есть дубли.
* Привести названия столбцов в соответствие с требованием питона.
* Есть выбросы в distanse.
* Есть пропуски.
* Есть NaN.
* Не правильно обозначенны типы данных.
* Дописать выводы!!

## 3. Подготовка данных

### 3.1 Основаня таблица 

In [ ]:
# Приводим к PEP-8 Названия столбов
df_main.columns = df_main.columns.str.lower().str.replace(' ', '_')

In [ ]:
# ПРОВЕРКА ОТРАБОТКИ КОДА
df_main.head(1)

*Вывод: всё нормально*

In [ ]:
# Заполняем Пропуски NaN
df_main.fillna(np.nan, inplace=True)

In [ ]:
# Приводим 'start_location' и 'end_location' к уникальным занчениям без повтроения.
def to_normal(means):
    means = means.lower()
    means = means.replace(' ', '')
    means = means.replace('ул', 'ул.')
    means = means.replace('ул..', 'ул.')
    means = means.replace('ул.', '')
    
    return means


df_main['start_location'] = df_main['start_location'].apply(to_normal)
df_main['end_location'] = df_main['end_location'].apply(to_normal)

In [ ]:
# Приводим 'start_district' и 'end_district' к уникальным занчениям без повтроения.
replc = {
    'Северо-Западный': 'северо-западный',
    'северо западный': 'северо-западный',
    'Северо Западный': 'северо-западныйй',
    'Юго-Восточный': 'юго-восточный',
    'Ленинский': 'ленинский',
    'Октябрьский': 'октябрьский',
    'Центральный': 'центральный',
    'Заречный': 'заречный',
    'Ленинский': 'ленинский',
    'Октябрьский': 'октябрьский',

}

# Проходим по столбцу 'категория' и заменяем значения согласно словарю замен
df_main['start_district'] = df_main['start_district'].replace(replc)
df_main['end_district'] = df_main['end_district'].replace(replc)

In [ ]:
# Проверяем что остались только уникальные занчения и потоврений больше нет.
print('\033[1m' + '\nВывод уникальных значений по каждому категориаьному признаку:'+ '\033[0m')    
df_object = df_main.select_dtypes(include='object').columns
for i in df_object:
        print('\033[1m' + '_'+ str(i) + '\033[0m')
        display(df_main[i].value_counts())

In [ ]:
pd.set_option('display.max_rows', 20)
# Находим явыне дубликаты и удаляем их 
dulc = df_main.duplicated(keep=False)
df_main.drop_duplicates(inplace=True)

#Явыне дубликаты
df_main[dulc].head(10)


In [ ]:
#Убираем выбросы
# Использование межквартильного размаха для удаления выбросов из столбца 'Distance'
Q1 = df_main['distance'].quantile(0.25)
Q3 = df_main['distance'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Удаление выбросов
df_main = df_main[(df_main['distance'] > lower_bound) & (df_main['distance'] < upper_bound)]

In [ ]:
# Заполняем пропуски в distance
# Преобразуем столбцы с датами в формат datetime
df_main['start_date'] = pd.to_datetime(df_main['start_date'])
df_main['end_date'] = pd.to_datetime(df_main['end_date'])

# Вычисляем время, затраченное на поездку, в минутах
df_main['travel_time'] = (df_main['end_date'] -
                          df_main['start_date']).dt.total_seconds() / 60 / 60

# Вычисляем среднюю скорость
average_speed = df_main['distance'].median() / df_main['travel_time'].median()

# Заполняем пропуски в столбце distance средним значением скорости
df_main['distance'].fillna(df_main['travel_time'] *
                           average_speed, inplace=True)

In [ ]:
# Заполняем пропущенное end_date

# Вычисляем медиану времени поездки и медиану расстояния
median_travel_time = df_main['travel_time'].median()
median_distance = df_main['distance'].median()

# Заполняем пропуски в столбце Travel Time медианным значением времени поездки
df_main['travel_time'].fillna(median_travel_time, inplace=True)

# Вычисляем медиану скорости
median_speed_m_per_min = median_distance / median_travel_time

# Заполняем пропуски в столбце end_date с использованием медианной скорости
df_main['end_date'].fillna(df_main['start_date'] + pd.to_timedelta(
    df_main['distance'] / median_speed_m_per_min, unit='min'), inplace=True)

# Округляем значения до ближайшего целого числа end_date
df_main['end_date'] = df_main['end_date'].dt.floor('min')

# Округляем значения до ближайшего целого числа start_date
df_main['start_date'] = df_main['start_date'].dt.floor('min')

# Удаляем временный столбец с временем поездки
df_main.drop('travel_time', axis=1, inplace=True)

In [ ]:
check_data(df_main)

### 3.2 Таблица с погодой

In [ ]:
# Приводим к PEP-8 Названия столбов
df_weather.columns = df_weather.columns.str.lower().str.replace(' ', '_')

In [ ]:
df_weather.head(1)

In [ ]:
# Заполняем Пропуски NaN
df_weather.fillna(np.nan, inplace=True)

In [ ]:
df_weather[df_weather.isna().any(axis=1)].head(10)

In [ ]:
# Преобразование столбца Datetime в формат datetime
df_weather["datetime"] = pd.to_datetime(df_weather["datetime"])

# Установка индекса DataFrame по столбцу Datetime
df_weather.set_index("datetime", inplace=True)

# Интерполяция только для пропущенных значений в столбце Temperature
df_weather["temperature"] = df_weather["temperature"].interpolate(method="linear", limit_direction="both")




In [ ]:
cat_graph(df_main, df_main.select_dtypes(include=[object]).columns)

### 3.3 Проверка работы с данными

#### 3.1 Основаня таблица 

In [ ]:
# Ещё раз применим функцию первичноо осмотра данных
check_data(df_main)

In [ ]:
cat_graph(df_main, df_main.select_dtypes(include=[object]).columns)

Выводы по работе с данными

#### 3.2 Таблица с погодой

In [ ]:
# Ещё раз применим функцию первичноо осмотра данных
check_data(df_weather)

## 4. Проведение расчётов и иследований

#### 4.1 Рассчёт итоговой стоимости поездки с учетом применения промокода на бесплатный старт.

In [ ]:
def get_num_of_day_on_week(date):
    import datetime
    zero_date = datetime.datetime(2023, 4, 22)
    delta_date = date - zero_date
    delta_days = delta_date.days + 5
    num_of_day = delta_days % 7
    # print(date, num_of_day)
    return num_of_day


def get_cost_of_minute(date):
    num_of_day = get_num_of_day_on_week(date)
    price = 0
    if 0 <= num_of_day <= 4:
        if 1 <= date.hour < 6:
            price = 3
        elif 6 <= date.hour < 10:
            price = 4
        elif 10 <= date.hour < 16:
            price = 5
        elif 16 <= date.hour < 22:
            price = 6
        elif 22 <= date.hour or date.hour < 1:
            price = 5
    else:
        if 1 <= date.hour < 6:
            price = 3
        elif 6 <= date.hour < 10:
            price = 4
        elif 10 <= date.hour < 16:
            price = 6
        elif 16 <= date.hour < 22:
            price = 7
        elif 22 <= date.hour or date.hour < 1:
            price = 6
    return price


def get_price(start_time, end_time, promo_used):
    price = get_cost_of_minute(start_time)
    minutes = (end_time - start_time).seconds // 60
    return (promo_used + 1) % 2 * 30 + minutes * price

In [ ]:
df_main['cost'] = df_main.apply(lambda row: get_price(row['start_date'], row['end_date'], row['promo']), axis=1)

In [ ]:
# Смотрим на результат.
df_main.head(10)

#### 4.2 Проверяем, стимулирует ли акция с бесплатным стартом по понедельникам спрос насамокаты. Окупается ли она.

In [ ]:
# demand scooter - спрос на самокаты (почасовой)
dem_scoot = df_main[['start_date', 'promo', 'cost']]
dem_scoot['frequency'] = 1
dem_scoot = dem_scoot.resample('1h', on='start_date').agg({'promo':'sum', 'cost': 'sum', 'frequency': 'count'})
dem_scoot.reset_index(inplace=True)
dem_scoot.head(3)

In [ ]:
# По понедельникам с 6:00 до 10:00 акция «Бесплатный старт» по промокоду.
dem_scoot_promo_time = dem_scoot[(6 <= dem_scoot['start_date'].dt.hour) 
                                 & (dem_scoot['start_date'].dt.hour <= 9)]
dem_scoot_promo_time = dem_scoot_promo_time.resample('1d', on='start_date').agg({'promo':'sum', 'cost': 'sum', 'frequency': 'sum'})
dem_scoot_promo_time.reset_index(inplace=True)
# В dem_scoot_promo_time хранится информация о спросе по дням во временное окно акции
dem_scoot_promo_time.head(3)

In [ ]:
monthes_translate = {4: 'апреле',
                     5: 'мае',
                     6: 'июне',
                     7: 'июле'}
monthes = dem_scoot_promo_time['start_date'].dt.month.unique()

fig, axs = plt.subplots(nrows=len(monthes), figsize=(16, 16))
fig.suptitle('Время с 6:00 по 10:00', fontweight='bold', fontsize=14)

for i, m in enumerate(monthes):
    dem_scoot_data = dem_scoot_promo_time[(dem_scoot_promo_time['start_date'].dt.month == m)]

    x_values = dem_scoot_data['start_date']
    y_values = dem_scoot_data['frequency']

    dates = matplotlib.dates.date2num(x_values)

    axs[i].plot_date(dates, y_values, color='#DBE2E9', linestyle='-')

    promo_rides = dem_scoot_data[dem_scoot_data['promo'] > 0]

    x_values = promo_rides['start_date']
    y_values = promo_rides['frequency']

    dates = matplotlib.dates.date2num(x_values)
    axs[i].plot_date(dates, y_values, label='с промокодом')
    

    no_promo_rides = dem_scoot_data[dem_scoot_data['promo'] == 0]

    x_values = no_promo_rides['start_date']
    y_values = no_promo_rides['frequency']

    dates = matplotlib.dates.date2num(x_values)
    axs[i].plot_date(dates, y_values, label='без промокода')

    axs[i].tick_params(axis='x', labelrotation=90)
    axs[i].legend(loc="upper left")
    axs[i].set_xlabel('Время')
    axs[i].set_ylabel('Спрос')
    axs[i].set_title(f'Спрос в {monthes_translate[m]}', fontweight='bold')
    axs[i].grid(color='#F1EDF2')
    axs[i].xaxis.set_major_locator(mdates.DayLocator(interval=1))
fig.tight_layout()

**Вывод акция с промокодом**

#### 4.3 Исследование корреляции между параметрами погодных условий.

#### 4.4 Таблица с почасовым спросом на самокаты.

In [ ]:
# demand scooter - спрос на самокаты (почасовой)
dem_scoot.head(5)

#### 4.5 Определяем необходимое количесво самокатов в каждой точке и в каждом районе.

In [ ]:
# locations это list всевозможных локаций(точек)
locations = set(df_main['start_location'].unique())
locations = locations.union(set(df_main['start_location'].unique()))
locations = list(locations)

In [ ]:
'''
Будем считать что каждый день в 6:00 на точках появляется необходимое количество самокатов,
иначе будет накапливаться минус в точках где откуда самокаты чаще уезжают(/ШУТКА: Завода производящего самокаты там нет/)

Время 6:00 выбрано потому что предполагаем что в этот час начинается движение, запускается цикл спроса 
(работники, студенты, ... направляются в свои учреждения)
'''
locations_traffic = dict()

for location in locations:
    df_start = df_main[df_main['start_location'] == location][['start_date', 'cost']]
    df_start['traffic'] = 1
    df_start.columns = ['date', 'cost', 'traffic']

    df_end = df_main[df_main['end_location'] == location][['end_date', 'cost']]
    df_end['traffic'] = -1
    df_end.columns = ['date', 'cost', 'traffic']
    df_location = pd.concat([df_start, df_end], ignore_index=True)
    df_location = df_location.sort_values(by='date')
    # df_location = df_location.resample('1h', on='date').agg({'traffic': 'sum'}).reset_index()
    df_location['traffic_cum'] = df_location.resample('d', on='date')['traffic'].cumsum()
    df_location['traffic_cum_cont'] = df_location['traffic'].cumsum()
    locations_traffic[location] = df_location
df_location.head(3)

In [ ]:
'''
Предполагаю что количество необходимых самокатов в точках зависит от дня недели.
Поэтому считаю минимум по траффику самокатов в каждой точке по каждому дню отдельно,
также считаю минимум за все время и среднее за все время по минимумам по дням недели в точке.

Минимальное будет браться среди отрицательных чисел, 
так как нам важно понять сколько самокатов необходимо для того чтобы каждый смог воспользоваться услугой.
Человек не может воспользоваться услугой когда число самокатов в точке отрицательно(нет самокатов).
'''
df_locations_minuse_traffic_stat = pd.DataFrame({'location': [], 
                                 'mean_min_at_1d': [], 
                                 'min_at_all_time_1d': [],
                                 'mean_min_at_2d': [], 
                                 'min_at_all_time_2d': [],
                                 'mean_min_at_3d': [], 
                                 'min_at_all_time_3d': [],
                                 'mean_min_at_4d': [], 
                                 'min_at_all_time_4d': [],
                                 'mean_min_at_5d': [], 
                                 'min_at_all_time_5d': [],
                                 'mean_min_at_6d': [], 
                                 'min_at_all_time_6d': [],
                                 'mean_min_at_7d': [], 
                                 'min_at_all_time_7d': [],
                                 'mean_min_at_all_time': [],
                                 'min_at_all_time': []})

'''
Посмотрим на количество самокатов которое остается в конце дня
'''
df_locations_remains_stat = pd.DataFrame({'location': [], 
                                 'mean_at_1d': [], 
                                 'max_at_all_time_1d': [],
                                 'mean_at_2d': [], 
                                 'max_at_all_time_2d': [],
                                 'mean_at_3d': [], 
                                 'max_at_all_time_3d': [],
                                 'mean_at_4d': [], 
                                 'max_at_all_time_4d': [],
                                 'mean_at_5d': [], 
                                 'max_at_all_time_5d': [],
                                 'mean_at_6d': [], 
                                 'max_at_all_time_6d': [],
                                 'mean_at_7d': [], 
                                 'max_at_all_time_7d': [],
                                 'mean_at_all_time': [],
                                 'max_at_all_time': []})

for location in locations_traffic:
    location_traffic_day = locations_traffic[location].resample('1d', origin='2023-04-01 06:00:00', on='date').agg({'traffic': 'sum', 'traffic_cum': 'min'})
    location_traffic_day.reset_index(inplace=True)
    location_traffic_day['num_day'] = location_traffic_day['date'].apply(get_num_of_day_on_week)

    # Для df_locations_minuse_traffic_stat
    location_stat = [location]
    for day in location_traffic_day['num_day'].unique():
        min_traffics_at_day = location_traffic_day[(location_traffic_day['num_day'] == day)
                                                    & (location_traffic_day['traffic_cum'] < 0)]['traffic_cum']
        mean_min_at_day = min_traffics_at_day.mean()
        min_at_all_time = min_traffics_at_day.min()
        location_stat.extend([mean_min_at_day, min_at_all_time])
    location_stat.append(location_traffic_day[location_traffic_day['traffic_cum'] < 0]['traffic_cum'].mean())
    location_stat.append(location_traffic_day[location_traffic_day['traffic_cum'] < 0]['traffic_cum'].min())
    df_locations_minuse_traffic_stat.loc[len(df_locations_minuse_traffic_stat)] = location_stat

    # Для df_locations_remains_stat
    location_stat = [location]
    for day in location_traffic_day['num_day'].unique():
        min_traffics_at_day = location_traffic_day[location_traffic_day['num_day'] == day]['traffic']
        mean_min_at_day = min_traffics_at_day.mean()
        min_at_all_time = min_traffics_at_day.max()
        location_stat.extend([mean_min_at_day, min_at_all_time])
    location_stat.append(location_traffic_day['traffic'].mean())
    location_stat.append(location_traffic_day['traffic'].max())
    df_locations_remains_stat.loc[len(df_locations_remains_stat)] = location_stat


df_locations_minuse_traffic_stat = df_locations_minuse_traffic_stat.fillna(0)
cols = df_locations_minuse_traffic_stat.columns.to_list()
cols.pop(0)
for col in cols:
    df_locations_minuse_traffic_stat[col] = df_locations_minuse_traffic_stat[col].astype(int)
df_locations_minuse_traffic_stat.sort_values(by='min_at_all_time', inplace=True)

df_locations_remains_stat = df_locations_remains_stat.fillna(0)
cols = df_locations_remains_stat.columns.to_list()
cols.pop(0)
for col in cols:
    df_locations_remains_stat[col] = df_locations_remains_stat[col].astype(int)
df_locations_remains_stat.sort_values(by='mean_at_all_time', inplace=True)

In [ ]:
df_locations_minuse_traffic_stat.head(3)

In [ ]:
df_locations_remains_stat.head(3)

In [ ]:
'''
В датафрейме информация о том сколько самокатов должно быть в точке утром в 6:00 для того чтобы полностью удовлетворить спрос
'''
df_necessary_quantity_scoots = df_locations_minuse_traffic_stat[['location', 'min_at_all_time']]
df_necessary_quantity_scoots.loc[:, 'min_at_all_time'] = df_necessary_quantity_scoots['min_at_all_time'].abs()
df_necessary_quantity_scoots.columns = ['location', 'necessary_quantity']
df_necessary_quantity_scoots = df_necessary_quantity_scoots.reset_index().drop('index', axis=1)
df_necessary_quantity_scoots.head(3)

In [ ]:
''' 
Попытка анализировать трафик в точках по наличию тренда,
строю регресию и сортирую точки по углу полученой линии 

'''
def is_trand(data):
    x = np.arange(0, len(data))
    y = np.array(data)

    z = np.polyfit(x, y, 1)
    return z[0], z[1]


df_loc_traf_cum_cont = pd.DataFrame({'location': [], 'traffic_cum_cont': [], 'mean_traffic_cum_cont': [], 'reg_tg': [], 'reg_b': []})
for location in locations:
    df_location_traffic = locations_traffic[location]

    res = is_trand(df_location_traffic['traffic_cum_cont'])
    df_loc_traf_cum_cont.loc[len(df_loc_traf_cum_cont)] = [location, 
                                                           df_location_traffic.loc[len(df_location_traffic) - 1, 'traffic_cum_cont'],
                                                           df_location_traffic['traffic_cum_cont'].mean(),
                                                           res[0],
                                                           res[1]]
x_ = 100
df_loc_traf_cum_cont['reg_dec'] = df_loc_traf_cum_cont['reg_tg'] * x_**2 + df_loc_traf_cum_cont['reg_b'] + x_
df_loc_traf_cum_cont.sort_values(by='reg_dec', inplace=True)
df_loc_traf_cum_cont.head(3)

In [ ]:
name_num_day = {
    0: 'пн',
    1: 'вт',
    2: 'ср',
    3: 'чт',
    4: 'пт',
    5: 'сб',
    6: 'вс'
}


month = 7
fig, axs = plt.subplots(nrows=3, figsize=(16, 16))
fig.suptitle(f'Трафик в {monthes_translate[month]} (trafic_1)', fontweight='bold', fontsize=14)

for i, location in enumerate(['спортивная', 'советская', 'зеленая']):
    df_location = locations_traffic[location]

    x = df_location[df_location['date'].dt.month == month]['date']
    y_returns = df_location[df_location['date'].dt.month == month]['traffic_cum']


    axs[i].fill_between(x, y_returns, 0, where=y_returns >= 0, facecolor='green', interpolate=True, alpha=0.7)
    axs[i].fill_between(x, y_returns, 0, where=y_returns <= 0, facecolor='red', interpolate=True, alpha=0.7)

    days = x.reset_index().set_index('date').resample('d').sum().reset_index().drop('index', axis=1)['date']
    xtickvals = [str(date.date().day) + "-" + name_num_day[get_num_of_day_on_week(date)] for date in days]
    

    axs[i].grid(alpha=0.2)
    axs[i].xaxis.set_major_locator(mdates.DayLocator(interval=1))
    axs[i].set_xticks(days)
    axs[i].set_xlim(days.iloc[0], days.iloc[len(days) - 1] + timedelta(days=1))
    axs[i].set_ylim(-20, 20)

    axs[i].set_xticklabels(xtickvals, rotation=90)
    
    axs[i].set_xlabel('Время')
    axs[i].set_ylabel('Трафик')
    axs[i].set_title(f'Улица {location.capitalize()}')

fig.tight_layout()
''' 
Ниже график trafic_1
'''

In [ ]:
name_num_day = {
    0: 'пн',
    1: 'вт',
    2: 'ср',
    3: 'чт',
    4: 'пт',
    5: 'сб',
    6: 'вс'
}

# streets = ['спортивная', 'советская', 'зеленая']
streets = df_loc_traf_cum_cont['location'].to_list()[:5] + ['советская'] + df_loc_traf_cum_cont['location'].to_list()[-5:]
fig, axs = plt.subplots(nrows=len(streets), figsize=(16, 6 * len(streets)))
fig.suptitle('Трафик (trafic_2)', fontweight='bold', fontsize=14)

for i, location in enumerate(streets):
    df_location = locations_traffic[location]

    x = df_location['date']
    y_returns = df_location['traffic'].cumsum()


    axs[i].fill_between(x, y_returns, 0, where=y_returns >= 0, facecolor='green', alpha=0.7)
    axs[i].fill_between(x, y_returns, 0, where=y_returns <= 0, facecolor='red', alpha=0.7)

    

    axs[i].grid(alpha=0.2)
    axs[i].xaxis.set_major_locator(mdates.DayLocator(interval=5))

    
    axs[i].set_xlabel('Время')
    axs[i].set_ylabel('Трафик')
    axs[i].set_title(f'Улица {location.capitalize()}')
    axs[i].tick_params(axis='x', labelrotation=90)

fig.tight_layout()
''' 
Ниже график trafic_2
'''


Для графиков были взяты топ-5 улиц сверху и снизу остсортированного датафрейма df_loc_traf_cum_cont
Датафрейм df_loc_traf_cum_cont был отсортирован по возрастанию площади под линией регрессии построенной по трафику.
Сделано это было для того чтобы определить точки с возраст./убыв. трендом количества самокатов. 
Далее были выведены графики для проверки соответствия
Улица 'советская' добавлена в качестве премера той улицы, которую не стоит использовать для балансировки.


In [ ]:
streets



Из датафрэйма df_locations_remains_stat отсортированного по возрастанию среднего по оставшимся самокатам в 6:00
Видно что самокаты в целом в конце дня если и остаются то в малом количестве, что делает не выгодным их транспортировку в другие точки,
а также из датафрэйма df_necessary_quantity_scoots видно что во все точки необходимо поставлять самокаты чтобы покрыть спрос
Получается что перераспредилить самокаты в конце дня не получится.
Возможно для перераспределения самокатов нужно выбирать разное время для каждой точки, тогда удасться покрыть спрос, так как в некоторые точки самокаты чаще приехжают,
а из других уезжают (например: 'спортивная', 'зеленая') -> Это видно из графика trafic_2 на улицу 'зеленая' чаще приезжают, поэтому если самокаты от туда не увозить,
то они будут копиться, к примеру их можно увозить на улицу 'спортивная'.

Но сказать что стабильно можно перенаправлять самокаты в конце дня с какой то улицы не получается, потому что есть много минимумов, которые видно из графика,
и предугадать когда можно увезти получится только при более детальном исследованиии и при отслеживании самокатов online

Пока что кажется что по этим данным решить задачу балансировки самокатов offline не получится.
Но можно сказать примерно из каких точек в какие можно перенаправить.
Для этого обратимся к графику выше.
По проведённому исследованию предлагаю делать балансировку самокатов из ['куйбышева', 'дачная', 'пионерская', 'нагорная', 'пушкина']
в ['мира', 'шевченко', 'заводская', 'матросова', 'вокзальная']


#### 4.6 Определим Топ-3 точки с самым высоким трафиком


Определим точки с самым высоким трафиком.
За трафик буду считать количество самокатов которое приезжает/уезжает из точки в час времени (кол-во/час).

Считаю что в расчёт нужно брать все часы, так как ночью у некоторых точек может повышаться спрос.
Если брать все часы, то среднее будет меньше чем если бы мы брали среднее лишь по дню, так как много часов простоя,
но это справедливо для всех точек.


In [ ]:

location_demand_h = dict()
df_location_mean_demand = pd.DataFrame({'location': [], 
                                        'mean_demand_h': [], 
                                        'mean_demand_d': [], 
                                        'median_demand_d': [], 
                                        'shapiro_pvalue': [],
                                        'mean_cum_cost_h': [],
                                        'cost_all_time': []})
for location in locations:
    df_location = locations_traffic[location].drop('traffic_cum', axis=1)
    df_location.loc[:, 'traffic'] = df_location['traffic'].abs()
    df_location_d = df_location.resample('1d', on='date').sum()
    df_location = df_location.resample('1h', on='date').sum()
    location_demand_h[location] = df_location
    df_location_mean_demand.loc[len(df_location_mean_demand)] = [location, 
                                                                 df_location['traffic'].mean(), 
                                                                 df_location_d['traffic'].mean(),
                                                                 df_location_d['traffic'].median(),
                                                                 round(shapiro(df_location['traffic']).pvalue, 4),
                                                                 df_location['cost'].mean(),
                                                                 df_location['cost'].sum()
                                                                 ]

df_location_mean_demand = df_location_mean_demand.sort_values(by='mean_demand_h', key=lambda x: -x).reset_index().drop('index', axis=1)
df_location_mean_demand.join(df_necessary_quantity_scoots.set_index('location'), on='location')
df_location_mean_demand.head(10)

Из таблицы видно что среднее ~ соответствует медиане, а также shapiro стремится к 0 (распред нормальное, можно судить по среднему)

In [ ]:
# Топ-3 точки с самым высоким трафиком

df_location_mean_demand.loc[:2]

In [ ]:
# луговая
df_location_demand_h = location_demand_h['луговая']
monthes = df_location_demand_h.index.month.unique()
fig, axs = plt.subplots(nrows=len(monthes), figsize=(16, 16))
for i, month in enumerate(monthes[:]):
    df_location_demand_h_m = df_location_demand_h[df_location_demand_h.index.month == month]
    x = df_location_demand_h_m.index
    y = df_location_demand_h_m['traffic']

    axs[i].plot_date(x, y, markersize=1)
    axs[i].fill_between(x, y, 0, facecolor='blue', alpha=0.7)
    axs[i].tick_params(axis='x', labelrotation=90)
    axs[i].xaxis.set_major_locator(mdates.DayLocator(interval=1))

    days = df_location_demand_h_m.resample('d').sum().index
    xtickvals = [str(date.date().day) + "-" + name_num_day[get_num_of_day_on_week(date)] for date in days]

    axs[i].set_xticks(days)
    axs[i].set_xlim(days[0], days[len(days) - 1] + timedelta(days=1))
    axs[i].set_ylim(0, 12)
    axs[i].set_xticklabels(xtickvals, rotation=90)

In [ ]:
'пушкина'
'''
дзержинского	0.691384
66	северная	0.691128
67	сосновая	0.688484
68	овражная	0.687967
69	зеленая	0.687526
70	пролетарская	0.686413
71	южная	0.686283
72	заречная	0.684932
73	совхозная	0.683513
74	коммунальная	0.683513
75	кооперативная	0.682573
76	нагорная	0.680780
77	молодежная	0.680066
78	дачная	0.679519
79	гоголя	0.679503
80	энтузиастов	0.676337
81	советская	0.674814
82	сиреневая	0.674399
83	железнодорожная	0.672750
84	солнечная	0.670813
85	дружбы	0.669983
86	куйбышева	0.667909
87	мира	0.667634
88	степная	0.660315
89	набережная
'''
df_location_demand_h = location_demand_h['гоголя']
monthes = df_location_demand_h.index.month.unique()
fig, axs = plt.subplots(nrows=4, figsize=(16, 10))
for i, month in enumerate(range(4, 7 + 1)):
    df_location_demand_h_m = df_location_demand_h[df_location_demand_h.index.month == month]
    x = df_location_demand_h_m.index
    y = df_location_demand_h_m['traffic']

    axs[i].plot_date(x, y, markersize=1)
    axs[i].fill_between(x, y, 0, facecolor='blue', alpha=0.7)
    axs[i].tick_params(axis='x', labelrotation=90)
    axs[i].xaxis.set_major_locator(mdates.DayLocator(interval=1))

    days = df_location_demand_h_m.resample('d').sum().index
    xtickvals = [str(date.date().day) + "-" + name_num_day[get_num_of_day_on_week(date)] for date in days]

    axs[i].set_xticks(days)
    axs[i].set_xlim(days[0], days[len(days) - 1] + timedelta(days=1))
    axs[i].set_ylim(0, 12)
    axs[i].set_xticklabels(xtickvals, rotation=90)

Из графиков точки с самым высоким средним спросом и самым низким можно увидить разницу

#### 4.7 Самые популярные направления перемещения

In [ ]:
df_directions = pd.DataFrame()
df_directions['direction_location'] = df_main['start_location'] + '->' + df_main['end_location']
df_directions['direction_ditsrict'] = df_main['start_district'] + '->' + df_main['end_district']

In [ ]:
df_directions['direction_location'].value_counts().head(10)

In [ ]:
df_directions['direction_ditsrict'].value_counts().head(10)


direction_location 
direction_district в топе находятся таки направления как 

## 5. Провека гипотез


#### 5.2 Проверка различия средней стоимости поездки по районам

In [ ]:
# Группировка данных по районам и вычисление средней стоимости поездки для каждого района
mean_costs_by_district = df_main.groupby('start_district')['cost'].mean()

In [ ]:
# Вывод средних значений стоимости поездок по районам
print("Средняя стоимость поездок по районам:")
print(mean_costs_by_district)

In [ ]:
# Проведение однофакторного дисперсионного анализа (ANOVA)
# Получение данных о стоимости поездок для каждого района
costs_by_district = []
for district in df_main['start_district'].unique():
    costs_by_district.append(df_main[df_main['start_district'] == district]['cost'])

# Проведение ANOVA
f_statistic, p_value = f_oneway(*costs_by_district)

# Вывод результатов ANOVA
print("\nРезультаты однофакторного дисперсионного анализа (ANOVA):")
print("F-статистика:", f_statistic)
print("p-значение:", p_value)

# Оценка статистической значимости
alpha = 0.05
if p_value < alpha:
    print("\nОтвергаем нулевую гипотезу: средние значения стоимости поездок по районам различаются.")
else:
    print("\nНе отвергаем нулевую гипотезу: нет статистически значимых различий в средней стоимости поездок по районам.")


**Вывод: Средняя стоимость поездки не отличается по районам**

#### 5.3 Проверка зависимости дня недели от спроса на самокты.

In [ ]:
# Добавление столбца для дня недели (0 - 4 будни, 5 - 6  выходные)
df_main['day_of_week'] = df_main['start_date'].dt.dayofweek

In [ ]:
# Агрегирование данных
daily_demand = df_main.groupby('day_of_week').size()

In [ ]:
# Визуализация результатов
plt.figure(figsize=(10, 6))
daily_demand.plot(kind='bar', color='skyblue')
plt.title('Спрос на самокаты по дням недели')
plt.xlabel('День недели')
plt.ylabel('Количество поездок')
plt.xticks(range(7), ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'])
plt.show()

**Вывод: Из графика видно, что на спрос влияет день недели. Следоватьельно гипотеза подтвердилась**

#### 5.4 Проверка длительности поездок по выходным и будням

In [ ]:
# Разделение данных на будни и выходные
weekday_trips = df_main[df_main['day_of_week'] < 5]
weekend_trips = df_main[df_main['day_of_week'] >= 5]

In [ ]:
# Вычисление длительности поездок в минутах
weekday_trips['duration'] = (weekday_trips['end_date'] - weekday_trips['start_date']).dt.total_seconds() / 60
weekend_trips['duration'] = (weekend_trips['end_date'] - weekend_trips['start_date']).dt.total_seconds() / 60

In [ ]:
# Вывод средних длительностей поездок
print("Средняя длительность поездок по будням:", weekday_trips['duration'].mean(), "минут")
print("Средняя длительность поездок по выходным:", weekend_trips['duration'].mean(), "минут")

**Вывод: В среднем поездки по времени одинакоыве, следоватьельно гипотеза опровегнута**

## 6 Регрессионное моделирование

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
dem_scoot.head(3)
dem_scoot_index_date = dem_scoot.set_index('start_date')
train, test = train_test_split(dem_scoot_index_date['frequency'], test_size=0.2, shuffle=False)

In [ ]:
model = SARIMAX(train, 
                order = (3, 1, 1), 
                seasonal_order = (2, 1, 1, 24))
result = model.fit()

In [ ]:
start = test.index[0]
 
# и закончится в конце тестового
end = test.index[-1]
  
# применим метод predict
predictions = result.predict(start, end)
predictions

In [ ]:
monthes = dem_scoot['start_date'].dt.month.unique()
fig, axs = plt.subplots(nrows=len(monthes), figsize=(16, 16))

for i, month in enumerate(monthes):
    # выведем три кривые (обучающая, тестовая выборка и тестовый прогноз)
    monthes_train = train.index.month.unique()
    monthes_test = test.index.month.unique()
    monthes_predictions = predictions.index.month.unique()
    if month in monthes_train:
        data = train[train.index.month == month]
        x = data.index
        y = data.values
        axs[i].plot(x, y, color='black')
    if month in monthes_test:
        data = test[test.index.month == month]
        x = data.index
        y = data.values
        axs[i].plot(x, y, color='green')
    if month in monthes_predictions:
        data = predictions[predictions.index.month == month]
        x = data.index
        y = data.values
        axs[i].plot(x, y, color='red')
    
    # days = x.reset_index().set_index('date').resample('d').sum().reset_index().drop('index', axis=1)['date']
    # xtickvals = [str(date.date().day) + "-" + name_num_day[get_num_of_day_on_week(date)] for date in days]
    

    # axs[i].grid(alpha=0.2)
    # axs[i].xaxis.set_major_locator(mdates.DayLocator(interval=1))
    # axs[i].set_xticks(days)
    # axs[i].set_xlim(days.iloc[0], days.iloc[len(days) - 1] + timedelta(days=1))
    # axs[i].set_ylim(-20, 20)

    # axs[i].set_xticklabels(xtickvals, rotation=90)
    
    # axs[i].set_xlabel('Время')
    # axs[i].set_ylabel('Трафик')
    # axs[i].set_title(f'Спрос в {}')


 
# рассчитаем MSE
print(mean_squared_error(test, predictions))
 
# и RMSE
print(np.sqrt(mean_squared_error(test, predictions)))